In [ ]:
import pandas as pd
from carsus.util import parse_selected_species
from carsus.io.cmfgen import CMFGENReader, CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser

class CustomAtomicReader_Tanaka():
    """
    Class for extracting levels and lines from the Japan Lithuania Atomic Group.

    Mimics the GFALLReader class.

    Attributes
    ----------
    levels : DataFrame
    lines : DataFrame

    """

    def __init__(self, data, priority=10):
        """
        Parameters
        ----------
        data : dict
            Dictionary containing one dictionary per species with
            keys `levels` and `lines`.

        priority: int, optional
            Priority of the current data source, by default 10.
        """
        self.priority = priority
        self._get_levels_lines(data)
        self.version = 1.0

    def _get_levels_lines(self, data):
        """Generates `levels` and `lines` DataFrames.

        Parameters
        ----------
        data : dict
            Dictionary containing one dictionary per species with
            keys `levels` and `lines`.
        """
        lvl_list = []
        lns_list = []
        for ion, parser in data.items():
            print(ion, parser)
            atomic_number = parse_selected_species(ion)[0][0]
            ion_charge = parse_selected_species(ion)[0][1]
            levels_data = pd.read_csv(parser["levels"], skiprows=1, sep = '\s+', header=None, names=range(20))
            levels = pd.DataFrame()
            levels['energy'] = levels_data.iloc[:,3]
            levels['j'] = 0.5*(levels_data.iloc[:,1]-1)
            levels['label'] = 'missing'
            levels['method'] = 'calc'
            levels['priority'] = 10
            levels['atomic_number'] = atomic_number
            levels['ion_charge'] = ion_charge
            levels['level_index'] = levels.index + 1
            levels = levels.set_index(["atomic_number", "ion_charge", 'level_index']).reindex()

            
            lines_data = pd.read_csv(parser["lines"], skiprows=1, sep = '\s+', header=None, names=range(20))
            lines = pd.DataFrame()
            lines['level_index_lower'] = lines_data.iloc[:,1]
            lines['level_index_upper'] = lines_data.iloc[:,0]
            lines['atomic_number'] = atomic_number
            lines['ion_charge'] = ion_charge
            lines['energy_lower'] = levels.iloc[lines.iloc[:,0]-1]['energy'].values
            lines['energy_upper'] = levels.iloc[lines.iloc[:,1]-1]['energy'].values
            lines['gf'] = lines_data.iloc[:,4]
            lines['gf'] = 10**lines['gf']
            lines['j_lower'] = levels.iloc[lines.iloc[:,0]-1]['j'].values
            lines['j_upper'] = levels.iloc[lines.iloc[:,1]-1]['j'].values
            lines['wavelength'] = lines_data.iloc[:,2]
            lines = lines.set_index(["atomic_number", "ion_charge", 'level_index_lower', 'level_index_upper']).reindex()
            levels = levels.reset_index()
            levels['level_index'] = levels['level_index'] - 1
            levels = levels.set_index(["atomic_number", "ion_charge", 'level_index']).reindex()
            lines = lines.reset_index()
            lines['level_index_lower'] = lines['level_index_lower'] - 1
            lines['level_index_upper'] = lines['level_index_upper'] - 1
            lines = lines.set_index(["atomic_number", "ion_charge", 'level_index_lower', 'level_index_upper']).reindex()

            display(levels)
            lvl_list.append(levels)
            display(lines)
            lns_list.append(lines)
        levels = pd.concat(lvl_list)
        lines = pd.concat(lns_list)
        self.levels = levels
        self.lines = lines
        return



In [ ]:
tanaka_reader = CustomAtomicReader_Tanaka(tanaka_data, priority=20)

from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-U')

from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-U')

from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', collisions=True, priority=20)

from carsus.io.output import TARDISAtomData


from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()
display(tanaka_reader.levels)

atom_data_with_cmfgen = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           tanaka_reader)
atom_data_with_cmfgen.to_hdf('kurucz_tanaka_except_Sr_Y_Zr_Ba3.h5')

[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Yb 12`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Yb 13`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Yb 14`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 6`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 7`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 9`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 10`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 13`. (ionization.py:167)
[  carsus.io.nist.ionization][WARNING] - Set `J=0` for ground state of species `Lu 14`. (ionization.py:167)
[  carsus.io.nist.ionization][W

ValueError: At based indexing on an integer index can only have integer indexers